## Einführung in die Regression - Lektion 1

#### Einordnung in den Kontext

✅ Es gibt viele Arten von Regressionsmethoden, und welche du wählst, hängt von der Frage ab, die du beantworten möchtest. Möchtest du beispielsweise die wahrscheinliche Körpergröße einer Person in einem bestimmten Alter vorhersagen, würdest du `lineare Regression` verwenden, da du nach einem **numerischen Wert** suchst. Wenn du hingegen herausfinden möchtest, ob eine bestimmte Küche als vegan betrachtet werden sollte oder nicht, suchst du nach einer **Kategorisierung**, und dafür würdest du `logistische Regression` verwenden. Mehr über logistische Regression wirst du später lernen. Überlege dir ein paar Fragen, die du an Daten stellen könntest, und welche dieser Methoden dafür am besten geeignet wäre.

In diesem Abschnitt wirst du mit einem [kleinen Datensatz über Diabetes](https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html) arbeiten. Stell dir vor, du möchtest eine Behandlung für Diabetespatienten testen. Machine-Learning-Modelle könnten dir dabei helfen, herauszufinden, welche Patienten besser auf die Behandlung ansprechen würden, basierend auf Kombinationen von Variablen. Selbst ein sehr einfaches Regressionsmodell könnte, wenn es visualisiert wird, Informationen über Variablen liefern, die dir bei der Organisation deiner theoretischen klinischen Studien helfen könnten.

Also, lass uns mit dieser Aufgabe beginnen!

<p >
   <img src="../../images/encouRage.jpg"
   width="630"/>
   <figcaption>Kunstwerk von @allison_horst</figcaption>

<!--![Kunstwerk von \@allison_horst](../../../../../../translated_images/encouRage.e75d5fe0367fb9136b78104baf4e2032a7622bc42a2bc34c0ad36c294eeb83f5.de.jpg)<br>Kunstwerk von @allison_horst-->


## 1. Laden unseres Werkzeugkastens

Für diese Aufgabe benötigen wir die folgenden Pakete:

-   `tidyverse`: Das [tidyverse](https://www.tidyverse.org/) ist eine [Sammlung von R-Paketen](https://www.tidyverse.org/packages), die darauf ausgelegt ist, Datenwissenschaft schneller, einfacher und unterhaltsamer zu machen!

-   `tidymodels`: Das [tidymodels](https://www.tidymodels.org/) Framework ist eine [Sammlung von Paketen](https://www.tidymodels.org/packages/) für Modellierung und maschinelles Lernen.

Sie können sie wie folgt installieren:

`install.packages(c("tidyverse", "tidymodels"))`

Das untenstehende Skript überprüft, ob Sie die für dieses Modul benötigten Pakete installiert haben, und installiert sie für Sie, falls einige fehlen.


In [2]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))
pacman::p_load(tidyverse, tidymodels)

Loading required package: pacman



Lassen Sie uns nun diese großartigen Pakete laden und in unserer aktuellen R-Sitzung verfügbar machen. (Dies dient nur zur Veranschaulichung, `pacman::p_load()` hat das bereits für Sie erledigt.)


In [ ]:
# load the core Tidyverse packages
library(tidyverse)

# load the core Tidymodels packages
library(tidymodels)


## 2. Der Diabetes-Datensatz

In dieser Übung werden wir unsere Regressionsfähigkeiten unter Beweis stellen, indem wir Vorhersagen auf einem Diabetes-Datensatz treffen. Der [Diabetes-Datensatz](https://www4.stat.ncsu.edu/~boos/var.select/diabetes.rwrite1.txt) umfasst `442 Stichproben` mit Daten zu Diabetes, darunter 10 Prädiktorvariablen: `Alter`, `Geschlecht`, `Body-Mass-Index`, `durchschnittlicher Blutdruck` und `sechs Messungen des Blutserums` sowie eine Zielvariable `y`: ein quantitativer Messwert für den Krankheitsverlauf ein Jahr nach der Ausgangsmessung.

|Anzahl der Beobachtungen|442|
|-------------------------|:---|
|Anzahl der Prädiktoren|Die ersten 10 Spalten sind numerische Prädiktoren|
|Zielvariable|Spalte 11 ist ein quantitativer Messwert für den Krankheitsverlauf ein Jahr nach der Ausgangsmessung|
|Informationen zu den Prädiktoren|- Alter in Jahren
||- Geschlecht
||- bmi Body-Mass-Index
||- bp durchschnittlicher Blutdruck
||- s1 tc, Gesamtserumcholesterin
||- s2 ldl, Low-Density-Lipoproteine
||- s3 hdl, High-Density-Lipoproteine
||- s4 tch, Gesamtcholesterin / HDL
||- s5 ltg, möglicherweise Logarithmus des Serumtriglyceridspiegels
||- s6 glu, Blutzuckerspiegel|


> 🎓 Denke daran, dies ist überwachtes Lernen, und wir benötigen eine benannte Zielvariable 'y'.

Bevor du Daten mit R bearbeiten kannst, musst du die Daten in den Speicher von R importieren oder eine Verbindung zu den Daten herstellen, die R für den Fernzugriff auf die Daten verwenden kann.

> Das [readr](https://readr.tidyverse.org/)-Paket, das Teil des Tidyverse ist, bietet eine schnelle und benutzerfreundliche Möglichkeit, rechteckige Daten in R einzulesen.

Lass uns nun den Diabetes-Datensatz von der folgenden URL laden: <https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html>

Außerdem werden wir eine Plausibilitätsprüfung unserer Daten mit `glimpse()` durchführen und die ersten 5 Zeilen mit `slice()` anzeigen.

Bevor wir weitermachen, möchten wir noch etwas vorstellen, das du oft in R-Code sehen wirst 🥁🥁: den Pipe-Operator `%>%`

Der Pipe-Operator (`%>%`) führt Operationen in logischer Reihenfolge aus, indem er ein Objekt an eine Funktion oder einen Ausdruck weiterleitet. Du kannst dir den Pipe-Operator so vorstellen, als würdest du in deinem Code "und dann" sagen.


In [ ]:
# Import the data set
diabetes <- read_table2(file = "https://www4.stat.ncsu.edu/~boos/var.select/diabetes.rwrite1.txt")


# Get a glimpse and dimensions of the data
glimpse(diabetes)


# Select the first 5 rows of the data
diabetes %>% 
  slice(1:5)

`glimpse()` zeigt uns, dass diese Daten 442 Zeilen und 11 Spalten enthalten, wobei alle Spalten den Datentyp `double` haben.

<br>

> `glimpse()` und `slice()` sind Funktionen aus [`dplyr`](https://dplyr.tidyverse.org/). Dplyr, ein Teil des Tidyverse, ist eine Grammatik für Datenmanipulation, die eine konsistente Reihe von Verben bereitstellt, um die häufigsten Herausforderungen bei der Datenmanipulation zu lösen.

<br>

Da wir nun die Daten haben, konzentrieren wir uns auf ein Merkmal (`bmi`), das wir für diese Übung verwenden möchten. Dafür müssen wir die gewünschten Spalten auswählen. Wie machen wir das?

[`dplyr::select()`](https://dplyr.tidyverse.org/reference/select.html) ermöglicht es uns, Spalten in einem Dataframe *auszuwählen* (und optional umzubenennen).


In [ ]:
# Select predictor feature `bmi` and outcome `y`
diabetes_select <- diabetes %>% 
  select(c(bmi, y))

# Print the first 5 rows
diabetes_select %>% 
  slice(1:10)

## 3. Trainings- und Testdaten

Es ist gängige Praxis im überwachten Lernen, die Daten in zwei Teilmengen aufzuteilen: einen (in der Regel größeren) Satz, mit dem das Modell trainiert wird, und einen kleineren "Rückhalte"-Satz, mit dem überprüft wird, wie gut das Modell funktioniert hat.

Da wir nun die Daten vorbereitet haben, können wir prüfen, ob eine Maschine helfen kann, eine logische Aufteilung zwischen den Zahlen in diesem Datensatz zu bestimmen. Wir können das [rsample](https://tidymodels.github.io/rsample/)-Paket verwenden, das Teil des Tidymodels-Frameworks ist, um ein Objekt zu erstellen, das die Informationen darüber enthält, *wie* die Daten aufgeteilt werden sollen. Anschließend können zwei weitere rsample-Funktionen verwendet werden, um die erstellten Trainings- und Testdatensätze zu extrahieren:


In [ ]:
set.seed(2056)
# Split 67% of the data for training and the rest for tesing
diabetes_split <- diabetes_select %>% 
  initial_split(prop = 0.67)

# Extract the resulting train and test sets
diabetes_train <- training(diabetes_split)
diabetes_test <- testing(diabetes_split)

# Print the first 3 rows of the training set
diabetes_train %>% 
  slice(1:10)

## 4. Trainieren eines linearen Regressionsmodells mit Tidymodels

Jetzt sind wir bereit, unser Modell zu trainieren!

In Tidymodels spezifizieren Sie Modelle mit `parsnip()`, indem Sie drei Konzepte angeben:

-   Der **Modelltyp** unterscheidet Modelle wie lineare Regression, logistische Regression, Entscheidungsbaum-Modelle und so weiter.

-   Der **Modus des Modells** umfasst gängige Optionen wie Regression und Klassifikation; einige Modelltypen unterstützen beide, während andere nur einen Modus haben.

-   Die **Engine des Modells** ist das rechnerische Werkzeug, das verwendet wird, um das Modell anzupassen. Oft sind dies R-Pakete, wie **`"lm"`** oder **`"ranger"`**.

Diese Modellinformationen werden in einer Modellspezifikation erfasst, also erstellen wir eine!


In [ ]:
# Build a linear model specification
lm_spec <- 
  # Type
  linear_reg() %>% 
  # Engine
  set_engine("lm") %>% 
  # Mode
  set_mode("regression")


# Print the model specification
lm_spec

Nachdem ein Modell *spezifiziert* wurde, kann das Modell mit der [`fit()`](https://parsnip.tidymodels.org/reference/fit.html)-Funktion `geschätzt` oder `trainiert` werden, typischerweise unter Verwendung einer Formel und einiger Daten.

`y ~ .` bedeutet, dass wir `y` als die vorhergesagte Größe/Zielvariable anpassen, erklärt durch alle Prädiktoren/Merkmale, also `.` (in diesem Fall haben wir nur einen Prädiktor: `bmi`).


In [ ]:
# Build a linear model specification
lm_spec <- linear_reg() %>% 
  set_engine("lm") %>%
  set_mode("regression")


# Train a linear regression model
lm_mod <- lm_spec %>% 
  fit(y ~ ., data = diabetes_train)

# Print the model
lm_mod

Aus den Modell-Ausgaben können wir die während des Trainings gelernten Koeffizienten erkennen. Sie repräsentieren die Koeffizienten der Ausgleichsgeraden, die den geringsten Gesamtfehler zwischen der tatsächlichen und der vorhergesagten Variablen liefert.
<br>

## 5. Vorhersagen für den Testdatensatz treffen

Nachdem wir nun ein Modell trainiert haben, können wir es verwenden, um die Krankheitsprogression y für den Testdatensatz mithilfe von [parsnip::predict()](https://parsnip.tidymodels.org/reference/predict.model_fit.html) vorherzusagen. Dies wird genutzt, um die Linie zwischen den Datenclustern zu ziehen.


In [ ]:
# Make predictions for the test set
predictions <- lm_mod %>% 
  predict(new_data = diabetes_test)

# Print out some of the predictions
predictions %>% 
  slice(1:5)

Woohoo! 💃🕺 Wir haben gerade ein Modell trainiert und es verwendet, um Vorhersagen zu treffen!

Beim Erstellen von Vorhersagen ist es in der tidymodels-Konvention üblich, immer ein Tibble/Data-Frame mit standardisierten Spaltennamen zu erzeugen. Dies erleichtert es, die Originaldaten und die Vorhersagen in einem nutzbaren Format zu kombinieren, um sie für nachfolgende Operationen wie das Plotten zu verwenden.

`dplyr::bind_cols()` verbindet effizient mehrere Data-Frames spaltenweise.


In [ ]:
# Combine the predictions and the original test set
results <- diabetes_test %>% 
  bind_cols(predictions)


results %>% 
  slice(1:5)

## 6. Modellierungsergebnisse visualisieren

Jetzt ist es an der Zeit, dies visuell darzustellen 📈. Wir erstellen ein Streudiagramm aller `y`- und `bmi`-Werte des Testdatensatzes und verwenden dann die Vorhersagen, um eine Linie an der passendsten Stelle zwischen den Datenclustern des Modells zu zeichnen.

R bietet mehrere Systeme zur Erstellung von Grafiken, aber `ggplot2` ist eines der elegantesten und vielseitigsten. Es ermöglicht dir, Grafiken durch **Kombination unabhängiger Komponenten** zu erstellen.


In [ ]:
# Set a theme for the plot
theme_set(theme_light())
# Create a scatter plot
results %>% 
  ggplot(aes(x = bmi)) +
  # Add a scatter plot
  geom_point(aes(y = y), size = 1.6) +
  # Add a line plot
  geom_line(aes(y = .pred), color = "blue", size = 1.5)

✅ Denk mal darüber nach, was hier genau passiert. Eine gerade Linie verläuft durch viele kleine Datenpunkte, aber was macht sie eigentlich genau? Kannst du erkennen, wie du diese Linie nutzen könntest, um vorherzusagen, wo ein neuer, noch nicht gesehener Datenpunkt in Bezug auf die y-Achse des Plots liegen sollte? Versuche, den praktischen Nutzen dieses Modells in Worte zu fassen.

Herzlichen Glückwunsch, du hast dein erstes lineares Regressionsmodell erstellt, eine Vorhersage damit gemacht und es in einem Plot dargestellt!



---

**Haftungsausschluss**:  
Dieses Dokument wurde mithilfe des KI-Übersetzungsdienstes [Co-op Translator](https://github.com/Azure/co-op-translator) übersetzt. Obwohl wir uns um Genauigkeit bemühen, weisen wir darauf hin, dass automatisierte Übersetzungen Fehler oder Ungenauigkeiten enthalten können. Das Originaldokument in seiner ursprünglichen Sprache sollte als maßgebliche Quelle betrachtet werden. Für kritische Informationen wird eine professionelle menschliche Übersetzung empfohlen. Wir übernehmen keine Haftung für Missverständnisse oder Fehlinterpretationen, die sich aus der Nutzung dieser Übersetzung ergeben.
